<a href="https://colab.research.google.com/github/whatnews72/----/blob/master/6%EC%A3%BC%EC%B0%A8_%EC%96%B4%ED%85%90%EC%85%98_%EB%A9%94%EC%BB%A4%EB%8B%88%EC%A6%98%EC%9D%98_%EC%9D%B4%ED%95%B4_%EC%8B%A4%EC%8A%B5_bahdanau_attention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import tensorflow as tf
import tensorflow.keras.backend as K

#1. bahdanau attention 만들어보기


# 먼저 numpy로 연산 구현

In [2]:
# hiddne state 값 준비
np.random.seed(0)
s = np.random.randn(16)  # 압축 차원 길이=16
h = np.random.randn(8, 16)  # encoder 시간 길이=8, 압축 차원 길이=16

In [3]:
# 가중치 생성
wa = np.random.randn(16, 16)
ua = np.random.randn(16, 16)
va = np.random.randn(1, 16)

In [4]:
# 계산1 - h와 s 각각을 가중치 곱하여 더하기
np.dot(h, ua.transpose()) + np.dot(s, wa.transpose())
print((np.dot(h, ua.transpose())+ np.dot(s, wa.transpose())))

[[  7.61717987  -7.18938783  -2.52083773  -1.29414899   7.93474544
   -0.1271671   -0.73063291  -7.71241991  -6.06493934   1.5649772
    4.78740838 -11.85840064  -5.94269757  -2.71069843  -4.07306779
    1.47724267]
 [  7.13079868  -7.41483643  -2.73950687   9.29952666  -3.98348742
  -10.79813258   2.15539554   4.64091217   9.74361292  11.13991559
  -11.9095806   -3.51880155   6.65654036  -6.01739336   2.03975548
  -13.71637056]
 [  1.90285712  -0.71423676  -3.88977638   5.94233621   0.2609294
   -2.03654225  -0.36482031  -4.26071705   6.52987709   7.59356316
   -5.72573936  -3.17054463   7.38326078  -2.85673656  -4.21502066
   -2.22219225]
 [  3.73116511  -3.46729739  -1.81626053   8.7347636    3.06134644
   -3.47390509   2.18275343  -2.14665946   5.23029824   3.58260949
   -2.04728874  -3.79624096   0.26560409   0.09563467  -1.69669981
   -3.81847252]
 [  2.90031077  -8.62192674  -3.86991289   0.42693164   1.30002176
   -0.81866129   1.84453331  -3.45712528   5.99108412   9.01609373


In [5]:
#계산2 - h와 s 각각을 가중치 곱하여 더한 결과에 tanh적용
np.tanh(np.dot(h, ua.transpose()) + np.dot(s, wa.transpose()))

array([[ 0.99999952, -0.99999886, -0.98715718, -0.8602093 ,  0.99999974,
        -0.12648601, -0.6234524 , -0.9999996 , -0.99998921,  0.91622313,
         0.9998611 , -1.        , -0.99998622, -0.99119698, -0.99942046,
         0.9009501 ],
       [ 0.99999872, -0.99999927, -0.9916878 ,  0.99999998, -0.99930679,
        -1.        ,  0.97350972,  0.99981382,  0.99999999,  1.        ,
        -1.        , -0.99824508,  0.99999669, -0.99998813,  0.96673129,
        -1.        ],
       [ 0.95648139, -0.61332672, -0.99916395,  0.99998621,  0.25516462,
        -0.96652039, -0.34945281, -0.99960177,  0.99999574,  0.99999949,
        -0.99997873, -0.99648144,  0.99999923, -0.99341935, -0.99956367,
        -0.97678399],
       [ 0.99885203, -0.99805486, -0.94846425,  0.99999995,  0.99562451,
        -0.99808037,  0.97490252, -0.97304911,  0.99994272,  0.99845517,
        -0.96722063, -0.99899205,  0.25952971,  0.09534418, -0.93499524,
        -0.99903587],
       [ 0.99396691, -0.99999994, -0

In [6]:
# 계산 3 - tanh 결과에 다중치 va 곱하기
np.dot(np.tanh(np.dot(h, ua.transpose()) + np.dot(s, wa.transpose())), va.transpose())

array([[ 0.50622235],
       [ 0.23123622],
       [-1.24593044],
       [ 1.08933605],
       [-0.56696991],
       [-2.58057185],
       [ 2.58568857],
       [-5.01320722]])

In [7]:
# softmax 정의
def softmax(x):
    return np.exp(x)/sum(np.exp(x))

In [8]:
# tanh에 va를 곱한 결과에 soxtmax를 적용하여 attention score 생성
softmax(np.dot(np.tanh(np.dot(h, ua.transpose()) + np.dot(s, wa.transpose())), va.transpose()))

array([[8.25330375e-02],
       [6.26906641e-02],
       [1.43112490e-02],
       [1.47866874e-01],
       [2.82193074e-02],
       [3.76747293e-03],
       [6.60280592e-01],
       [3.30803444e-04]])

In [9]:
# attention score와 encoder hidden state 가중합
np.sum(softmax(np.dot(np.tanh(np.dot(h, ua.transpose()) + np.dot(s, wa.transpose())), va.transpose()))*h, axis=0)

array([-0.53047481,  1.08989808, -0.4134333 ,  0.50146525,  0.39927329,
        0.06619121,  0.58819996,  0.55271133,  0.50512843, -1.07621964,
        0.13134064,  0.90750036, -0.48981229, -0.06244843, -0.39363165,
        1.30814755])

# 같은 연산을 Keras로 구현


In [10]:
# 위에서 생성한 값들을 tensor로 변환
h = tf.convert_to_tensor(h)
s = tf.convert_to_tensor(s.reshape(1, -1))
wa = tf.convert_to_tensor(wa)
ua = tf.convert_to_tensor(ua)
va = tf.convert_to_tensor(va)

In [11]:
 # tanh - va 곱까지 구현
e = K.dot(K.tanh(K.dot(h, K.transpose(ua)) + K.dot(s, K.transpose(wa))), K.transpose(va))
print(e)

tf.Tensor(
[[ 0.50622235]
 [ 0.23123622]
 [-1.24593044]
 [ 1.08933605]
 [-0.56696991]
 [-2.58057185]
 [ 2.58568857]
 [-5.01320722]], shape=(8, 1), dtype=float64)


In [12]:
 # softmax 적용한 attention 가중치 생성 구현
 a = K.softmax(e, axis=0)
 print(a)
 print(K.sum(a, axis=0))

tf.Tensor(
[[8.25330375e-02]
 [6.26906641e-02]
 [1.43112490e-02]
 [1.47866874e-01]
 [2.82193074e-02]
 [3.76747293e-03]
 [6.60280592e-01]
 [3.30803444e-04]], shape=(8, 1), dtype=float64)
tf.Tensor([1.], shape=(1,), dtype=float64)


In [13]:
# encoder hidden state 가중합 구현
c = K.sum(a*h, axis=0)
print(c)

tf.Tensor(
[-0.53047481  1.08989808 -0.4134333   0.50146525  0.39927329  0.06619121
  0.58819996  0.55271133  0.50512843 -1.07621964  0.13134064  0.90750036
 -0.48981229 -0.06244843 -0.39363165  1.30814755], shape=(16,), dtype=float64)
